In [ ]:
# ==================== IMPORTS ====================
import numpy as np
import pandas as pd
import os
import json
import glob
from datetime import datetime
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.dpi'] = 100

print("✓ All libraries imported successfully!")

In [ ]:
# ==================== SETUP DIRECTORIES ====================
print("\n" + "="*80)
print("SETUP - DETECTING ENVIRONMENT")
print("="*80)

# Detect environment
try:
    output_dir = "/kaggle/working/comprehensive_analysis"
    os.makedirs(output_dir, exist_ok=True)
    is_kaggle = True
    print("✓ Running on Kaggle")
except:
    output_dir = os.path.join(r"c:\Users\almir\ai-privacy\backend", "comprehensive_analysis")
    os.makedirs(output_dir, exist_ok=True)
    is_kaggle = False
    print("✓ Running locally")

print(f"Output directory: {output_dir}")

In [ ]:
# ==================== LOAD ALL RESULTS ====================
print("\n" + "="*80)
print("LOADING ALL EXPERIMENTAL RESULTS")
print("="*80)

results = {'baseline': None, 'fl': None, 'dp': None}

# Helper function to find files
def find_file(filename, kaggle_pattern=None, local_paths=None):
    """Try to find file in Kaggle input or local paths"""
    if is_kaggle and kaggle_pattern:
        paths = glob.glob(kaggle_pattern)
        if paths:
            return paths[0]
    
    if local_paths:
        for path in local_paths:
            if os.path.exists(path):
                return path
    
    return None

# Load Baseline Results
baseline_path = find_file(
    'research_results.json',
    kaggle_pattern='/kaggle/input/*/research_results.json',
    local_paths=[
        r'c:\Users\almir\ai-privacy\backend\models_research\research_results.json',
        r'c:\Users\almir\ai-privacy\backend\models_research_fl_dp\research_results.json'
    ]
)

if baseline_path:
    with open(baseline_path, 'r') as f:
        results['baseline'] = json.load(f)
    print(f"✓ Baseline results loaded from: {baseline_path}")
else:
    print("⚠️  Baseline results not found")

# Load FL Results
fl_path = find_file(
    'fl_adult_results.json',
    kaggle_pattern='/kaggle/input/*/fl_adult_results.json',
    local_paths=[
        r'c:\Users\almir\ai-privacy\backend\models_fl_adult\fl_adult_results.json',
        r'c:\Users\almir\ai-privacy\backend\models_fl_continue\fl_adult_results.json'
    ]
)

if fl_path:
    with open(fl_path, 'r') as f:
        results['fl'] = json.load(f)
    print(f"✓ FL results loaded from: {fl_path}")
else:
    print("⚠️  FL results not found")

# Load DP Results
dp_path = find_file(
    'dp_continue_results.json',
    kaggle_pattern='/kaggle/input/*/dp_continue_results.json',
    local_paths=[
        r'c:\Users\almir\ai-privacy\backend\models_research_dp_continue\dp_continue_results.json',
        r'c:\Users\almir\ai-privacy\backend\models_research_dp\dp_results.json'
    ]
)

if dp_path:
    with open(dp_path, 'r') as f:
        results['dp'] = json.load(f)
    print(f"✓ DP results loaded from: {dp_path}")
else:
    print("⚠️  DP results not found")

print("\n" + "-"*80)
print("Available results:")
for key, value in results.items():
    status = "✓" if value else "✗"
    print(f"  {status} {key.upper()}")

In [ ]:
# ==================== EXTRACT BASELINE DATA ====================
print("\n" + "="*80)
print("EXTRACTING BASELINE DATA")
print("="*80)

baseline_summary = []

if results['baseline']:
    for dataset in ['diabetes', 'adult']:
        for model in ['LR', 'FNN']:
            data = results['baseline']['baseline_results'][dataset][model]
            baseline_summary.append({
                'Dataset': dataset,
                'Model': model,
                'Method': 'Baseline',
                'Accuracy': data['accuracy']['mean'] * 100,
                'Std': data['accuracy']['std'] * 100,
                'Min': data['accuracy']['min'] * 100,
                'Max': data['accuracy']['max'] * 100,
                'F1': data['f1']['mean'] * 100,
                'F1_Std': data['f1']['std'] * 100
            })
    
    baseline_df = pd.DataFrame(baseline_summary)
    print("\n" + baseline_df.to_string(index=False))
else:
    print("⚠️  Skipping - baseline results not available")
    baseline_df = pd.DataFrame()

In [ ]:
# ==================== EXTRACT FL DATA ====================
print("\n" + "="*80)
print("EXTRACTING FEDERATED LEARNING DATA")
print("="*80)

fl_summary = []

if results['fl']:
    for config_key, config_data in results['fl']['federated_learning'].items():
        fl_summary.append({
            'Dataset': config_data['dataset'],
            'Model': config_data['model'],
            'Method': f"FL-{config_data['aggregation']}",
            'Aggregation': config_data['aggregation'],
            'Accuracy': config_data['accuracy']['mean'] * 100,
            'Std': config_data['accuracy']['std'] * 100,
            'Min': config_data['accuracy']['min'] * 100,
            'Max': config_data['accuracy']['max'] * 100,
            'F1': config_data['f1']['mean'] * 100,
            'F1_Std': config_data['f1']['std'] * 100
        })
    
    fl_df = pd.DataFrame(fl_summary)
    print("\n" + fl_df.to_string(index=False))
else:
    print("⚠️  Skipping - FL results not available")
    fl_df = pd.DataFrame()

In [ ]:
# ==================== EXTRACT DP DATA ====================
print("\n" + "="*80)
print("EXTRACTING DIFFERENTIAL PRIVACY DATA")
print("="*80)

dp_summary = []

if results['dp']:
    for config_key, config_data in results['dp']['differential_privacy'].items():
        dp_summary.append({
            'Dataset': config_data['dataset'],
            'Model': config_data['model'],
            'Method': f"DP-ε{config_data['target_epsilon']}",
            'Epsilon': config_data['target_epsilon'],
            'Actual_Epsilon': config_data['actual_epsilon'],
            'Accuracy': config_data['accuracy']['mean'] * 100,
            'Std': config_data['accuracy']['std'] * 100,
            'Min': config_data['accuracy']['min'] * 100,
            'Max': config_data['accuracy']['max'] * 100,
            'F1': config_data['f1']['mean'] * 100,
            'F1_Std': config_data['f1']['std'] * 100
        })
    
    dp_df = pd.DataFrame(dp_summary)
    print("\n" + dp_df.to_string(index=False))
else:
    print("⚠️  Skipping - DP results not available")
    dp_df = pd.DataFrame()

In [ ]:
# ==================== STATISTICAL COMPARISONS ====================
print("\n" + "="*80)
print("STATISTICAL COMPARISONS - T-TESTS")
print("="*80)

comparisons = []

if results['baseline']:
    # FL vs Baseline
    if results['fl']:
        print("\n📊 Federated Learning vs Baseline:")
        for config_key, fl_config in results['fl']['federated_learning'].items():
            dataset = fl_config['dataset']
            model = fl_config['model']
            baseline_key = f"{dataset}_{model}"
            
            if baseline_key in results['baseline']['baseline_results'][dataset]:
                baseline_data = results['baseline']['baseline_results'][dataset][model]
                baseline_acc = baseline_data['accuracy']['mean'] * 100
                baseline_all = baseline_data['all_accuracies']
                
                fl_acc = fl_config['accuracy']['mean'] * 100
                fl_all = fl_config['all_accuracies']
                
                t_stat, p_value = stats.ttest_ind(baseline_all, fl_all)
                
                comparisons.append({
                    'Comparison': f"FL-{fl_config['aggregation']} vs Baseline",
                    'Dataset': dataset,
                    'Model': model,
                    'Method_Acc': fl_acc,
                    'Baseline_Acc': baseline_acc,
                    'Difference': fl_acc - baseline_acc,
                    't_stat': t_stat,
                    'p_value': p_value,
                    'Significant': 'Yes' if p_value < 0.05 else 'No'
                })
    
    # DP vs Baseline
    if results['dp']:
        print("\n📊 Differential Privacy vs Baseline:")
        for config_key, dp_config in results['dp']['differential_privacy'].items():
            dataset = dp_config['dataset']
            model = dp_config['model']
            
            if dataset in results['baseline']['baseline_results']:
                baseline_data = results['baseline']['baseline_results'][dataset][model]
                baseline_acc = baseline_data['accuracy']['mean'] * 100
                baseline_all = baseline_data['all_accuracies']
                
                dp_acc = dp_config['accuracy']['mean'] * 100
                dp_all = dp_config['all_accuracies']
                
                t_stat, p_value = stats.ttest_ind(baseline_all, dp_all)
                
                comparisons.append({
                    'Comparison': f"DP-ε{dp_config['target_epsilon']} vs Baseline",
                    'Dataset': dataset,
                    'Model': model,
                    'Method_Acc': dp_acc,
                    'Baseline_Acc': baseline_acc,
                    'Difference': dp_acc - baseline_acc,
                    't_stat': t_stat,
                    'p_value': p_value,
                    'Significant': 'Yes' if p_value < 0.05 else 'No'
                })

if comparisons:
    comparison_df = pd.DataFrame(comparisons)
    print("\n" + comparison_df.to_string(index=False))
    
    # Save comparisons
    comparison_df.to_csv(os.path.join(output_dir, 'statistical_comparisons.csv'), index=False)
    print(f"\n✓ Saved: statistical_comparisons.csv")
else:
    print("\n⚠️  No comparisons possible - baseline results not available")
    comparison_df = pd.DataFrame()

In [ ]:
# ==================== VISUALIZATION 1: OVERALL COMPARISON ====================
print("\n" + "="*80)
print("VISUALIZATION 1: Overall Performance Comparison")
print("="*80)

if not baseline_df.empty or not fl_df.empty or not dp_df.empty:
    fig, axes = plt.subplots(2, 2, figsize=(18, 12))
    fig.suptitle('Comprehensive Privacy-Preserving ML Comparison', fontsize=16, fontweight='bold')
    
    datasets = ['diabetes', 'adult']
    models = ['LR', 'FNN']
    
    for idx, (dataset, model) in enumerate([(d, m) for d in datasets for m in models]):
        ax = axes[idx // 2, idx % 2]
        
        plot_data = []
        labels = []
        colors = []
        
        # Baseline
        if not baseline_df.empty:
            base = baseline_df[(baseline_df['Dataset'] == dataset) & (baseline_df['Model'] == model)]
            if not base.empty:
                plot_data.append((base['Accuracy'].values[0], base['Std'].values[0]))
                labels.append('Baseline')
                colors.append('red')
        
        # FL methods
        if not fl_df.empty:
            fl_subset = fl_df[(fl_df['Dataset'] == dataset) & (fl_df['Model'] == model)]
            for _, row in fl_subset.iterrows():
                plot_data.append((row['Accuracy'], row['Std']))
                labels.append(row['Aggregation'])
                colors.append('blue')
        
        # DP methods (just show a few key epsilon values)
        if not dp_df.empty:
            dp_subset = dp_df[(dp_df['Dataset'] == dataset) & (dp_df['Model'] == model)]
            # Select representative epsilon values
            for eps in [0.5, 3.0, 10.0]:
                dp_eps = dp_subset[dp_subset['Epsilon'] == eps]
                if not dp_eps.empty:
                    plot_data.append((dp_eps['Accuracy'].values[0], dp_eps['Std'].values[0]))
                    labels.append(f'DP-ε{eps}')
                    colors.append('green')
        
        if plot_data:
            x_pos = np.arange(len(plot_data))
            accs = [d[0] for d in plot_data]
            stds = [d[1] for d in plot_data]
            
            ax.bar(x_pos, accs, yerr=stds, capsize=5, alpha=0.7, color=colors)
            ax.set_xticks(x_pos)
            ax.set_xticklabels(labels, rotation=45, ha='right')
            ax.set_ylabel('Accuracy (%)')
            ax.set_title(f'{dataset.upper()} - {model}')
            ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    viz1_path = os.path.join(output_dir, '1_overall_comparison.png')
    plt.savefig(viz1_path, dpi=300, bbox_inches='tight')
    print(f"✓ Saved: 1_overall_comparison.png")
    plt.show()
else:
    print("⚠️  No data available for visualization")

In [ ]:
# ==================== VISUALIZATION 2: FL AGGREGATION METHODS ====================
print("\n" + "="*80)
print("VISUALIZATION 2: FL Aggregation Methods Comparison")
print("="*80)

if not fl_df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('Federated Learning: Aggregation Methods Comparison', fontsize=16, fontweight='bold')
    
    for idx, dataset in enumerate(['diabetes', 'adult']):
        ax = axes[idx]
        
        for model in ['LR', 'FNN']:
            subset = fl_df[(fl_df['Dataset'] == dataset) & (fl_df['Model'] == model)]
            
            if not subset.empty:
                x = range(len(subset))
                ax.errorbar(x, subset['Accuracy'], yerr=subset['Std'], 
                           marker='o', capsize=5, label=model, linewidth=2, markersize=8)
        
        # Add baseline if available
        if not baseline_df.empty:
            for model, color in [('LR', 'blue'), ('FNN', 'orange')]:
                base = baseline_df[(baseline_df['Dataset'] == dataset) & (baseline_df['Model'] == model)]
                if not base.empty:
                    ax.axhline(y=base['Accuracy'].values[0], color=color, 
                              linestyle='--', alpha=0.5, label=f'{model} Baseline')
        
        ax.set_xticks(range(5))
        ax.set_xticklabels(['FedAvg', 'FedProx', 'q-FedAvg', 'SCAFFOLD', 'FedAdam'], rotation=45, ha='right')
        ax.set_ylabel('Accuracy (%)')
        ax.set_title(f'{dataset.upper()}')
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    viz2_path = os.path.join(output_dir, '2_fl_aggregation_comparison.png')
    plt.savefig(viz2_path, dpi=300, bbox_inches='tight')
    print(f"✓ Saved: 2_fl_aggregation_comparison.png")
    plt.show()
else:
    print("⚠️  FL results not available")

In [ ]:
# ==================== VISUALIZATION 3: DP PRIVACY-ACCURACY TRADEOFF ====================
print("\n" + "="*80)
print("VISUALIZATION 3: DP Privacy-Accuracy Tradeoff")
print("="*80)

if not dp_df.empty:
    fig, axes = plt.subplots(1, 2, figsize=(16, 6))
    fig.suptitle('Differential Privacy: Privacy-Accuracy Tradeoff', fontsize=16, fontweight='bold')
    
    for idx, dataset in enumerate(['diabetes', 'adult']):
        ax = axes[idx]
        
        for model in ['LR', 'FNN']:
            subset = dp_df[(dp_df['Dataset'] == dataset) & (dp_df['Model'] == model)]
            
            if not subset.empty:
                subset_sorted = subset.sort_values('Epsilon')
                ax.errorbar(subset_sorted['Epsilon'], subset_sorted['Accuracy'], 
                           yerr=subset_sorted['Std'], marker='o', capsize=5, 
                           label=model, linewidth=2, markersize=8)
        
        # Add baseline if available
        if not baseline_df.empty:
            for model, color in [('LR', 'blue'), ('FNN', 'orange')]:
                base = baseline_df[(baseline_df['Dataset'] == dataset) & (baseline_df['Model'] == model)]
                if not base.empty:
                    ax.axhline(y=base['Accuracy'].values[0], color=color, 
                              linestyle='--', alpha=0.5, label=f'{model} Baseline')
        
        ax.set_xlabel('Privacy Budget (ε)')
        ax.set_ylabel('Accuracy (%)')
        ax.set_title(f'{dataset.upper()}')
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.set_xscale('log')
    
    plt.tight_layout()
    viz3_path = os.path.join(output_dir, '3_dp_privacy_accuracy_tradeoff.png')
    plt.savefig(viz3_path, dpi=300, bbox_inches='tight')
    print(f"✓ Saved: 3_dp_privacy_accuracy_tradeoff.png")
    plt.show()
else:
    print("⚠️  DP results not available")

In [ ]:
# ==================== VISUALIZATION 4: ACCURACY LOSS HEATMAP ====================
print("\n" + "="*80)
print("VISUALIZATION 4: Accuracy Loss from Baseline (Heatmap)")
print("="*80)

if not comparison_df.empty:
    # Pivot for heatmap
    heatmap_data = comparison_df.pivot_table(
        index='Comparison', 
        columns=['Dataset', 'Model'], 
        values='Difference'
    )
    
    fig, ax = plt.subplots(figsize=(12, 10))
    sns.heatmap(heatmap_data, annot=True, fmt='.2f', cmap='RdYlGn', center=0, 
                ax=ax, cbar_kws={'label': 'Accuracy Difference (%)'})
    ax.set_title('Accuracy Loss/Gain from Baseline (Negative = Loss)', fontsize=14, fontweight='bold')
    ax.set_ylabel('Method')
    ax.set_xlabel('Dataset - Model')
    
    plt.tight_layout()
    viz4_path = os.path.join(output_dir, '4_accuracy_loss_heatmap.png')
    plt.savefig(viz4_path, dpi=300, bbox_inches='tight')
    print(f"✓ Saved: 4_accuracy_loss_heatmap.png")
    plt.show()
else:
    print("⚠️  Comparison data not available")

In [ ]:
# ==================== SAVE SUMMARY TABLES ====================
print("\n" + "="*80)
print("SAVING SUMMARY TABLES")
print("="*80)

# Combine all results
all_results = []
if not baseline_df.empty:
    all_results.append(baseline_df)
if not fl_df.empty:
    all_results.append(fl_df)
if not dp_df.empty:
    all_results.append(dp_df)

if all_results:
    combined_df = pd.concat(all_results, ignore_index=True)
    combined_df.to_csv(os.path.join(output_dir, 'all_results_summary.csv'), index=False)
    print(f"✓ Saved: all_results_summary.csv ({len(combined_df)} rows)")

# Save individual summaries
if not baseline_df.empty:
    baseline_df.to_csv(os.path.join(output_dir, 'baseline_summary.csv'), index=False)
    print(f"✓ Saved: baseline_summary.csv")

if not fl_df.empty:
    fl_df.to_csv(os.path.join(output_dir, 'fl_summary.csv'), index=False)
    print(f"✓ Saved: fl_summary.csv")

if not dp_df.empty:
    dp_df.to_csv(os.path.join(output_dir, 'dp_summary.csv'), index=False)
    print(f"✓ Saved: dp_summary.csv")

print("\n" + "="*80)
print("✅ COMPREHENSIVE ANALYSIS COMPLETE")
print("="*80)
print(f"\nAll outputs saved to: {output_dir}")
print("\nGenerated files:")
for f in os.listdir(output_dir):
    print(f"  - {f}")